# Propose centroid replacing the masters with BERT siamese

In [1]:
# import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
# Optimize the use of GPUs
# https://datascience.stackexchange.com/questions/23895/multi-gpu-in-keras
# https://keras.io/getting-started/faq/#how-can-i-run-a-keras-model-on-multiple-gpus
# https://stackoverflow.com/questions/56316451/how-to-use-specific-gpus-in-keras-for-multi-gpu-training

In [3]:
from __future__ import print_function, division

In [4]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [5]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras import optimizers

In [6]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [7]:
MAX_SEQUENCE_LENGTH_T = 20 # 100
MAX_SEQUENCE_LENGTH_D = 20 # 500
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000
'''
    Configuration
'''
epochs = 1000
freeze_train = .1 # 10% with freeze weights
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

### Parse bugs preproprecessed

In [8]:
# Domain to use
DOMAIN = 'openoffice'
METHOD = 'deepQL_weights_{}'.format(epochs)
PREPROCESSING = 'bert'
TOKEN = 'bert'
# Dataset paths
DIR = 'data/processed/{}/{}'.format(DOMAIN, PREPROCESSING)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Glove embeddings
GLOVE_DIR='data/embed'
# Save model
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [9]:
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
model_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [10]:
from keras_bert import load_vocabulary

token_dict = load_vocabulary(vocab_path)

In [11]:
"Total vocabulary: {}".format(len(token_dict))

'Total vocabulary: 30522'

In [12]:
baseline = Baseline(DOMAIN, DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D,
                   token_dict['[CLS]'], token_dict['[SEP]'])
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [13]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

#### Loading bug ids in memory

In [14]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


98070

#### Dicionário de títulos e descrições

In [15]:
%%time

experiment.load_bugs(TOKEN)
len(baseline.sentence_dict)


CPU times: user 9.8 s, sys: 923 ms, total: 10.7 s
Wall time: 10.6 s


#### Hashing bugs by buckets

In [16]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [17]:
%%time

experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

Reading train data
Reading bug ids
CPU times: user 10min 12s, sys: 86.1 ms, total: 10min 12s
Wall time: 10min 12s


In [18]:
baseline.train_data[:10]

[[6036, 35039],
 [26311, 15091],
 [5142, 34523],
 [8323, 9594],
 [76676, 88775],
 [59725, 71546],
 [121756, 116964],
 [32936, 30911],
 [15257, 16297],
 [99719, 106395]]

#### Recovery bug ids from train

In [19]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

### Export the corpus train

In [20]:
if EXTRACT_CORPUS:
    corpus = []
    export_file = open(os.path.join(DIR, 'corpus_train.txt'), 'w')
    for bug_id in tqdm(baseline.bug_set):
        bug = baseline.bug_set[bug_id]
        title = bug['title']
        desc = bug['description']
        export_file.write("{}\n{}\n".format(title, desc))
    export_file.close()

In [21]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '2\n',
 'bug_status': '1\n',
 'component': '56\n',
 'creation_ts': '2005-03-12 17:35:00 +0000',
 'delta_ts': '2005-03-14 13:12:20 +0000',
 'description': '[CLS] hello very big bug i think . here is what i do : i open a s ##xi file into o ##o 2 . 0 ##bc ( french ) , on windows or linux . this file already contains a slide master loaded from a st ##i template . the file " export test o ##o ##2 in ##it . s ##xi " is an example . then i modify a little bit the file in o ##o 2 . 0 ##bc ( just add a word ) , and save it in s ##xi format . then i open it in o ##o 1 . 1 . 3 . it looks good . i modify it a bit ( add a word again ) and save it . then i re ##open it in o ##o 1 . 1 . 3 . it \' s completely broken ! it seems the slide master contains two outline text zones . the file " export test o ##o ##2 end . s ##xi " is what i get . philippe [SEP]',
 'description_segment': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [22]:
"Train ", len(baseline.dup_sets_train)

('Train ', 14508)

### Train ids

In [23]:
import random

# data - path
# batch_size - 128
# n_neg - 1
def batch_iterator(self, retrieval, model, data, dup_sets, bug_ids, 
                   batch_size, n_neg, issues_by_buckets, TRIPLET_HARD=False, FLOATING_PADDING=False):
    # global train_data
    # global self.dup_sets
    # global self.bug_ids
    # global self.bug_set

    random.shuffle(data)

    batch_features = {'title' : [], 'desc' : [], 'info' : []}
    n_train = len(data)

    batch_triplets, batch_bugs_anchor, batch_bugs_pos, batch_bugs_neg, batch_bugs = [], [], [], [], []

    all_bugs = list(issues_by_buckets.keys())
    buckets = retrieval.buckets

    for offset in range(batch_size):
        anchor, pos = data[offset][0], data[offset][1]
        batch_bugs_anchor.append(anchor)
        batch_bugs_pos.append(pos)
        batch_bugs.append(anchor)
        batch_bugs.append(pos)
        #batch_bugs += dup_sets[anchor]

    for anchor, pos in zip(batch_bugs_anchor, batch_bugs_pos):
        while True:
            neg = self.get_neg_bug(anchor, buckets[issues_by_buckets[anchor]], issues_by_buckets, all_bugs)
            bug_anchor = self.bug_set[anchor]
            bug_pos = self.bug_set[pos]
            if neg not in self.bug_set:
                continue
            batch_bugs.append(neg)
            batch_bugs_neg.append(neg)
            bug_neg = self.bug_set[neg]
            break
        
        # triplet bug and master
        batch_triplets.append([anchor, pos, neg])
    
    random.shuffle(batch_bugs)
    title_ids = np.full((len(batch_bugs), MAX_SEQUENCE_LENGTH_T), 0)
    description_ids = np.full((len(batch_bugs), MAX_SEQUENCE_LENGTH_D), 0)
    for i, bug_id in enumerate(batch_bugs):
        bug = self.bug_set[bug_id]
        self.read_batch_bugs(batch_features, bug, index=i, title_ids=title_ids, description_ids=description_ids)

    batch_features['title'] = { 'token' : np.array(batch_features['title']), 'segment' : title_ids }
    batch_features['desc'] = { 'token' : np.array(batch_features['desc']), 'segment' : description_ids }
    batch_features['info'] = np.array(batch_features['info'])
    
    sim = np.asarray([issues_by_buckets[bug_id] for bug_id in batch_bugs])

    input_sample = {}

    input_sample = { 'title' : batch_features['title'], 
                        'description' : batch_features['desc'], 
                            'info' : batch_features['info'] }

    return batch_triplets, input_sample, sim #sim

In [24]:
# %%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_sim = batch_iterator(baseline, retrieval, None, 
                                                                                      baseline.train_data, 
                                                                                      baseline.dup_sets_train,
                                                                                      bug_train_ids,
                                                                                      batch_size_test, 1,
                                                                                      issues_by_buckets)

validation_sample = [valid_input_sample['title']['token'], valid_input_sample['title']['segment'], 
                   valid_input_sample['description']['token'], valid_input_sample['description']['segment'],
                   valid_input_sample['info'], valid_sim]

# Categorical columns
number_of_columns_info = valid_input_sample['info'].shape[1]
# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title']['token'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description']['token'].shape[1]

In [25]:
valid_input_sample['title']['token'].shape, \
valid_input_sample['description']['token'].shape, \
valid_input_sample['title']['segment'].shape, \
valid_input_sample['description']['segment'].shape, \
valid_input_sample['info'].shape, valid_sim.shape

((384, 20), (384, 20), (384, 20), (384, 20), (384, 738), (384,))

### Validar entrada

In [26]:
# %%time 

#baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5)

In [27]:
"Test ", len(baseline.test_data)

('Test ', 8265)

## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

In [28]:
from keras.initializers import RandomUniform, RandomNormal, Ones

### BERT

https://github.com/CyberZHG/keras-bert

In [29]:
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import compile_model, get_model
from keras.layers import GlobalAveragePooling1D

def bert_model(MAX_SEQUENCE_LENGTH, name):
    layer_num = 8
#     model = load_trained_model_from_checkpoint(
#             config_path,
#             model_path,
#             training=True,
#             trainable=True,
#             seq_len=MAX_SEQUENCE_LENGTH,
#     )
    model = load_trained_model_from_checkpoint(
        config_path,
        model_path,
        training=True,
        use_adapter=True,
        seq_len=MAX_SEQUENCE_LENGTH,
        trainable=['Encoder-{}-MultiHeadSelfAttention-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-MultiHeadSelfAttention-Norm'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Norm'.format(i + 1) for i in range(layer_num)],
    )
#     model = get_model(
#         token_num=len(token_dict),
#         head_num=10,
#         transformer_num=layer_num,
#         embed_dim=100,
#         feed_forward_dim=100,
#         seq_len=MAX_SEQUENCE_LENGTH,
#         pos_num=MAX_SEQUENCE_LENGTH,
#         dropout_rate=0.05,
#     )
    compile_model(model)
    inputs = model.inputs[:2]
    outputs = model.get_layer('Encoder-{}-FeedForward-Norm'.format(layer_num)).output
    #outputs = model.get_layer('Extract').output
    outputs = GlobalAveragePooling1D()(outputs)
    outputs = Dense(300, activation='tanh')(outputs)
    
    model = Model(inputs, outputs, name='FeatureBERTGenerationModel{}'.format(name))
    
    return model

### MLP

In [30]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 300
    
    for units in [64, 32]:
        layer = Dense(units, activation='tanh', kernel_initializer='random_uniform')(info_input)
    
    layer = Dense(input_size, activation='tanh')(info_input)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [31]:
def pairwise_distance(feature, squared=False):
    """Computes the pairwise distance matrix with numerical stability.

    output[i, j] = || feature[i, :] - feature[j, :] ||_2

    Args:
      feature: 2-D Tensor of size [number of data, feature dimension].
      squared: Boolean, whether or not to square the pairwise distances.

    Returns:
      pairwise_distances: 2-D Tensor of size [number of data, number of data].
    """
    pairwise_distances_squared = math_ops.add(
        math_ops.reduce_sum(math_ops.square(feature), axis=[1], keepdims=True),
        math_ops.reduce_sum(
            math_ops.square(array_ops.transpose(feature)),
            axis=[0],
            keepdims=True)) - 2.0 * math_ops.matmul(feature,
                                                    array_ops.transpose(feature))

    # Deal with numerical inaccuracies. Set small negatives to zero.
    pairwise_distances_squared = math_ops.maximum(pairwise_distances_squared, 0.0)
    # Get the mask where the zero distances are at.
    error_mask = math_ops.less_equal(pairwise_distances_squared, 0.0)

    # Optionally take the sqrt.
    if squared:
        pairwise_distances = pairwise_distances_squared
    else:
        pairwise_distances = math_ops.sqrt(
            pairwise_distances_squared + math_ops.to_float(error_mask) * 1e-16)

    # Undo conditionally adding 1e-16.
    pairwise_distances = math_ops.multiply(
        pairwise_distances, math_ops.to_float(math_ops.logical_not(error_mask)))

    num_data = array_ops.shape(feature)[0]
    # Explicitly set diagonals to zero.
    mask_offdiagonals = array_ops.ones_like(pairwise_distances) - array_ops.diag(
        array_ops.ones([num_data]))
    pairwise_distances = math_ops.multiply(pairwise_distances, mask_offdiagonals)
    return pairwise_distances

def masked_maximum(data, mask, dim=1):
    """Computes the axis wise maximum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the maximum.

    Returns:
      masked_maximums: N-D `Tensor`.
        The maximized dimension is of size 1 after the operation.
    """
    axis_minimums = math_ops.reduce_min(data, dim, keepdims=True)
    masked_maximums = math_ops.reduce_max(
        math_ops.multiply(data - axis_minimums, mask), dim,
        keepdims=True) + axis_minimums
    return masked_maximums

def masked_minimum(data, mask, dim=1):
    """Computes the axis wise minimum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the minimum.

    Returns:
      masked_minimums: N-D `Tensor`.
        The minimized dimension is of size 1 after the operation.
    """
    axis_maximums = math_ops.reduce_max(data, dim, keepdims=True)
    masked_minimums = math_ops.reduce_min(
        math_ops.multiply(data - axis_maximums, mask), dim,
        keepdims=True) + axis_maximums
    return masked_minimums

In [32]:
## required for semi-hard triplet loss:
import tensorflow as tf
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes

def triplet_loss(vects):
    margin = 1.
    labels = vects[:, :1]
 
    labels = tf.cast(labels, dtype='int32')

    embeddings = tf.cast(vects[:, 1:], dtype='float32')

    ### Code from Tensorflow function [tf.contrib.losses.metric_learning.triplet_semihard_loss] starts here:
    
    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    # lshape=array_ops.shape(labels)
    # assert lshape.shape == 1
    # labels = array_ops.reshape(labels, [lshape[0], 1])

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    # Compute the mask.
    pdist_matrix_tile = array_ops.tile(pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and(
        array_ops.tile(adjacency_not, [batch_size, 1]),
        math_ops.greater(
            pdist_matrix_tile, array_ops.reshape(
                array_ops.transpose(pdist_matrix), [-1, 1])))
    mask_final = array_ops.reshape(
        math_ops.greater(
            math_ops.reduce_sum(
                math_ops.cast(mask, dtype=dtypes.float32), 1, keepdims=True),
            0.0), [batch_size, batch_size])
    mask_final = array_ops.transpose(mask_final)

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast(mask, dtype=dtypes.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = array_ops.reshape(
        masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose(negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = array_ops.tile(
        masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])
    semi_hard_negatives = array_ops.where(
        mask_final, negatives_outside, negatives_inside)

    loss_mat = math_ops.add(margin, pdist_matrix - semi_hard_negatives)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(
        array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)

    semi_hard_triplet_loss_distance = math_ops.truediv(
        math_ops.reduce_sum(
            math_ops.maximum(
                math_ops.multiply(loss_mat, mask_positives), 0.0)),
        num_positives,
        name='triplet_semihard_loss')
    
    ### Code from Tensorflow function semi-hard triplet loss ENDS here.
    return semi_hard_triplet_loss_distance

def quintet_loss(inputs):
    margin = 1.
    labels = inputs[:, :1]
 
    labels = tf.cast(labels, dtype='int32')

    embeddings = inputs[:, 1:]

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)
    
    mask_negatives = adjacency_not
    
    # Include the anchor to positives
    mask_positives_centroids = math_ops.cast(adjacency, dtype=dtypes.float32)

#     return mask_positives

   # pos 
    embed_pos = tf.matmul(mask_positives_centroids, embeddings)
    num_of_pos = tf.reduce_sum(mask_positives_centroids, axis=1, keepdims=True)
    centroid_embed_pos = tf.math.xdivy(embed_pos, num_of_pos)
    labels_pos = tf.cast(labels, dtype=dtypes.float32)
    # negs
    embed_neg = tf.matmul(mask_negatives, embeddings)
    num_of_neg = tf.reduce_sum(mask_negatives, axis=1, keepdims=True)
    centroid_embed_neg = tf.math.xdivy(embed_neg, num_of_neg)

#     return mask_positives_centroids
    i = tf.constant(0)
    batch_centroid_matrix = tf.Variable([])
    batch_centroid_matrix_neg = tf.Variable([])
    batch_centroid_matrix_all = tf.Variable([])
    def iter_centroids(i, batch_centroid_matrix, batch_centroid_matrix_neg, batch_centroid_matrix_all):
        # centroid pos
        mask_positives_batch = tf.reshape(tf.gather(mask_positives, i), (-1, 1))
        centroid_pos = tf.gather(centroid_embed_pos, i)
        
        centroid_embed = tf.repeat([centroid_pos], repeats=[batch_size], axis=0)
        new_batch_centroid_pos = mask_positives_batch * centroid_embed
        new_batch_embeddings = tf.cast(tf.logical_not(tf.cast(mask_positives_batch, 'bool')), 'float32') * embeddings 
        new_batch = tf.reduce_sum([new_batch_centroid_pos, new_batch_embeddings], axis=0, keepdims=True)[0]
        
        vects_new_batch = tf.concat([labels_pos, new_batch], axis=1)
        TL_new_batch = triplet_loss(vects_new_batch)
        batch_centroid_matrix = tf.concat([batch_centroid_matrix, [TL_new_batch]], axis=0) 
        
        # centroid neg
        centroid_neg = tf.gather(centroid_embed_neg, i)
        mask_negatives_batch = tf.reshape(tf.gather(mask_negatives, i), (-1, 1))
        
        centroid_embed = tf.repeat([centroid_neg], repeats=[batch_size], axis=0)
        new_batch_centroid_neg = mask_negatives_batch * centroid_embed
        new_batch_embeddings = tf.cast(tf.logical_not(tf.cast(mask_negatives_batch, 'bool')), 'float32') * embeddings 
        new_batch = tf.reduce_sum([new_batch_centroid_neg, new_batch_embeddings], axis=0, keepdims=True)[0]
        
        vects_new_batch = tf.concat([labels_pos, new_batch], axis=1)
        TL_new_batch = triplet_loss(vects_new_batch)
        batch_centroid_matrix_neg = tf.concat([batch_centroid_matrix_neg, [TL_new_batch]], axis=0) 
        
        # centroid pos and neg
        new_batch_centroids = tf.reduce_sum([new_batch_centroid_pos, new_batch_centroid_neg], axis=0, keepdims=True)[0]
        vects_new_batch_centroids = tf.concat([labels_pos, new_batch_centroids], axis=1)
        TL_new_batch = triplet_loss(vects_new_batch_centroids)
        batch_centroid_matrix_all = tf.concat([batch_centroid_matrix_all, [TL_new_batch]], axis=0) 
        
        return [tf.add(i, 1), batch_centroid_matrix, batch_centroid_matrix_neg, batch_centroid_matrix_all]
    _, batch_centroid_matrix, batch_centroid_matrix_neg, batch_centroid_matrix_all = tf.while_loop(lambda i, a, b, c: i<batch_size // 3, 
                                        iter_centroids, 
                                        [i, batch_centroid_matrix, batch_centroid_matrix_neg, batch_centroid_matrix_all],
                                       shape_invariants=[i.get_shape(),
                                                   tf.TensorShape([None]), tf.TensorShape([None]), tf.TensorShape([None])])

    TL_anchor_w = 1.0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
    TL_pos_w = 1.0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
    TL_neg_w = 1.0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
    TL_centroid_w = 1.0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
    
#     tl_weights = tf.truediv(num_of_pos, tf.reduce_max(num_of_pos))
#     tl_w = tl_weights # tf.random_uniform_initializer(minval=0.0, maxval=tl_weights)(shape=[1])
#     TL_pos_weighted = tf.reshape(batch_centroid_matrix, (-1, 1)) * tl_w
#     TL_pos = tf.truediv(tf.reduce_sum(TL_pos_weighted), tf.reduce_sum(tl_w))

    TL = triplet_loss(inputs)
    TL_pos = tf.reduce_mean(batch_centroid_matrix)
    TL_neg = tf.reduce_mean(batch_centroid_matrix_neg) #triplet_loss(vects_neg)
    TL_centroid = tf.reduce_mean(batch_centroid_matrix_all) # triplet_loss(vects_centroids)

    sum_of_median = tf.reduce_sum([TL * TL_anchor_w, TL_pos * TL_pos_w, TL_neg * TL_neg_w, TL_centroid * TL_centroid_w]) #  
    sum_of_weigths = TL_anchor_w + TL_pos_w + TL_neg_w + TL_centroid_w
    weigthed_median = tf.truediv(sum_of_median, sum_of_weigths)    
    return tf.cast([weigthed_median, TL_anchor_w, TL_pos_w, TL_neg_w, 
                    TL_centroid_w, TL, TL_pos, TL_neg, TL_centroid], 
                   dtype=dtypes.float32)


def custom_loss(y_true, y_pred):
    return tf.reduce_mean(y_pred[0])

def TL_w_anchor(y_true, y_pred):
    return tf.reduce_mean(y_pred[1])
def TL_w_pos(y_true, y_pred):
    return tf.reduce_mean(y_pred[2])
def TL_w_neg(y_true, y_pred):
    return tf.reduce_mean(y_pred[3])
def TL_w_centroid(y_true, y_pred):
    return tf.reduce_mean(y_pred[4])
def TL(y_true, y_pred):
    return tf.reduce_mean(y_pred[5])
def TL_pos(y_true, y_pred):
    return tf.reduce_mean(y_pred[6])
def TL_neg(y_true, y_pred):
    return tf.reduce_mean(y_pred[7])
def TL_centroid(y_true, y_pred):
    return tf.reduce_mean(y_pred[8])

### Propose

In [33]:
from keras.layers import concatenate, Add, Lambda, merge, Average, Maximum
from keras.optimizers import Adam, Nadam

def siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d, name):
  
    # Title
    bug_t_token = Input(shape = (sequence_length_t, ), name = 'title_token_{}'.format(name))
    bug_t_segment = Input(shape = (sequence_length_t, ), name = 'title_segment_{}'.format(name))
    # Description
    bug_d_token = Input(shape = (sequence_length_d, ), name = 'desc_token_{}'.format(name))
    bug_d_segment = Input(shape = (sequence_length_d, ), name = 'desc_segment_{}'.format(name))
    # Categorical
    bug_i = Input(shape = (sequence_length_info, ), name = 'info_{}'.format(name))
    
    bug_t_feat = title_feature_model([bug_t_token, bug_t_segment])
    bug_d_feat = desc_feature_model([bug_d_token, bug_d_segment])
    bug_i_feat = categorical_feature_model(bug_i)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = concatenate([bug_i_feat, bug_t_feat, bug_d_feat], name = 'merge_features_{}'.format(name))
    
    bug_feature_model = Model(inputs=[bug_t_token, bug_t_segment, bug_d_token, bug_d_segment, bug_i], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [34]:
def max_margin_objective(encoded_anchor, decay_lr=1):
    
    input_labels = Input(shape=(1,), name='input_label')    # input layer for labels
    inputs = np.concatenate([encoded_anchor.input, [input_labels]], -1).tolist()
    
    encoded_anchor = encoded_anchor.output
    
    output = concatenate([input_labels, encoded_anchor])  # concatenating the labels + embeddings
    
    output = Lambda(quintet_loss, name='quintet_loss')(output)
    
    similarity_model = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

    #optimizer = Nadam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.01)
    # optimizer = Adam(lr=1e-3 * decay_lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

    # setup the optimization process 
    similarity_model.compile(optimizer='adam', loss=custom_loss, metrics=[TL_w_anchor, TL_w_pos, TL_w_neg, TL_w_centroid,
                                                                         TL, TL_pos, TL_neg, TL_centroid]) 
    # metrics=[pos_distance, neg_distance, custom_margin_loss]

    return similarity_model

In [35]:
import _pickle as pickle
def save_loss(result):
    with open(os.path.join(DIR,'{}_log.pkl'.format(METHOD)), 'wb') as f:
        pickle.dump(result, f)
    print("=> result saved!")

In [37]:
# Domain to use
METHOD = 'deepQL_weights_{}'.format(limit_train)
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

In [36]:
%%time
import keras

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# Feature models
'''
    cnn_model
    lstm_model
    mlp_model
'''
title_feature_model = bert_model(MAX_SEQUENCE_LENGTH_T, 'Title')
desc_feature_model = bert_model(MAX_SEQUENCE_LENGTH_D, 'Description')
categorical_feature_model = mlp_model(number_of_columns_info)

# Similarity model
encoded_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'in')

similarity_model = max_margin_objective(encoded_anchor, decay_lr=1)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()

'''
    Experiment
'''
result = { 'train' : [], 'test' : [] }
limit_train = int(epochs * freeze_train) # 10% de 1000 , 100 epocas
print("Total of ", limit_train)
for epoch in range(limit_train):
    batch_triplet_train, \
        train_input_sample, train_sim = batch_iterator(baseline, retrieval, encoded_anchor, baseline.train_data, 
                                                       baseline.dup_sets_train, bug_train_ids, 
                                                           batch_size, 1, issues_by_buckets, TRIPLET_HARD=False)
    train_batch = [train_input_sample['title']['token'], train_input_sample['title']['segment'], 
                   train_input_sample['description']['token'], train_input_sample['description']['segment'],
                   train_input_sample['info'], train_sim]
    
#     if epoch == 10:
#         similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=0.1)
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    h_validation = similarity_model.test_on_batch(x=validation_sample, y=valid_sim)
    
    # save results
    result['train'].append(h)
    result['test'].append(h_validation)
    
    if( (epoch+1) % 10 == 0 or (epoch+1 == limit_train) ):
        save_loss(result)
    
    if (epoch+1 == limit_train): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets, 
                                                               bug_train_ids, method='bert')
        print(("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}\n" +
               "TL_w: {:.2f}, TL_pos_w: {:.2f}, TL_neg_w: {:.2f}, TL_centroid_w: {:.2f}\n" + 
                "TL: {:.2f}, TL_pos: {:.2f}, TL_neg: {:.2f}, TL_centroid: {:.2f}, " +
              "recall@25: {:.2f}").format(epoch+1, h[0], h_validation[0], h[1], h[2], h[3], 
                                          h[4], h[5], h[6], h[7], h[8], recall))
    else:
        print(("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}\n" +
               "TL_w: {:.2f}, TL_pos_w: {:.2f}, TL_neg_w: {:.2f}, TL_centroid_w: {:.2f}\n" + 
              "TL: {:.2f}, TL_pos: {:.2f}, TL_neg: {:.2f}, TL_centroid: {:.2f}").format(
            epoch+1, h[0], h_validation[0], h[1], h[2], h[3], h[4], h[5], h[6], h[7], h[8]))
    loss = h[0]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

# experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
# experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 738)          0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 20)           0                                            
__________________________________

/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "




Epoch: 1 Loss: 2.24, Loss_test: 1.11
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.73, TL_pos: 0.54, TL_neg: 6.97, TL_centroid: 0.71
Epoch: 2 Loss: 1.10, Loss_test: 1.03
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.73, TL_pos: 0.65, TL_neg: 2.62, TL_centroid: 0.41
Epoch: 3 Loss: 1.17, Loss_test: 0.95
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.67, TL_pos: 0.58, TL_neg: 2.84, TL_centroid: 0.58
Epoch: 4 Loss: 1.88, Loss_test: 0.86
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.60, TL_pos: 0.41, TL_neg: 5.77, TL_centroid: 0.74
Epoch: 5 Loss: 1.28, Loss_test: 0.77
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.49, TL_pos: 0.38, TL_neg: 3.60, TL_centroid: 0.65
Epoch: 6 Loss: 1.32, Loss_test: 0.68
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.49, TL_pos: 0.37, TL_neg: 3.69, TL_centroid: 0.74
Epoch: 7 Loss: 1.06, Loss_test: 0.60
TL_w: 1.00, TL_pos_

Epoch: 51 Loss: 0.12, Loss_test: 0.10
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.05, TL_pos: 0.05, TL_neg: 0.05, TL_centroid: 0.33
Epoch: 52 Loss: 0.13, Loss_test: 0.10
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.07, TL_pos: 0.07, TL_neg: 0.07, TL_centroid: 0.32
Epoch: 53 Loss: 0.09, Loss_test: 0.10
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.04, TL_pos: 0.04, TL_neg: 0.05, TL_centroid: 0.24
Epoch: 54 Loss: 0.15, Loss_test: 0.10
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.07, TL_pos: 0.07, TL_neg: 0.11, TL_centroid: 0.34
Epoch: 55 Loss: 0.11, Loss_test: 0.09
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.07, TL_pos: 0.07, TL_neg: 0.06, TL_centroid: 0.25
Epoch: 56 Loss: 0.12, Loss_test: 0.09
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.05, TL_pos: 0.05, TL_neg: 0.09, TL_centroid: 0.29
Epoch: 57 Loss: 0.11, Loss_test: 0.09
TL_w: 1.00, TL

In [38]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/openoffice/bert/exported_rank_deepQL_weights_100.txt'

In [39]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [40]:
experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(limit_train)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(limit_train)), verbose=1)

Saved model 'modelos/model_bert_preprocessing_deepQL_weights_100_feature_100epochs_64batch(openoffice).h5' to disk


In [41]:
len(result['train']), len(result['test'])

(100, 100)

In [42]:
model = similarity_model.get_layer('concatenate_1')
output = Lambda(quintet_loss, name='quintet_loss_unfreezed')(model.output)
inputs = similarity_model.inputs
model = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

# setup the optimization process 
model.compile(optimizer='adam', loss=custom_loss, metrics=[TL_w_anchor, TL_w_pos, TL_w_neg, TL_w_centroid,
                                                                     TL, TL_pos, TL_neg, TL_centroid])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 738)          0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 20)           0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 20)           0                                            
__________________________________________________________________________________________________
desc_segme

In [46]:
# Domain to use
METHOD = 'deepQL_weights_{}'.format(epochs)
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

In [43]:
end_train = epochs - limit_train
for epoch in range(limit_train, end_train):
    batch_triplet_train, \
        train_input_sample, train_sim = batch_iterator(baseline, retrieval, model, baseline.train_data, 
                                                       baseline.dup_sets_train, bug_train_ids, 
                                                           batch_size, 1, issues_by_buckets, TRIPLET_HARD=False)
    train_batch = [train_input_sample['title']['token'], train_input_sample['title']['segment'], 
                   train_input_sample['description']['token'], train_input_sample['description']['segment'],
                   train_input_sample['info'], train_sim]
    

    h = model.train_on_batch(x=train_batch, y=train_sim)
    h_validation = model.test_on_batch(x=validation_sample, y=valid_sim)
    
    # save results
    result['train'].append(h)
    result['test'].append(h_validation)
    
    if( (epoch+1) % 10 == 0 or (epoch+1 == end_train )):
        save_loss(result)
    
    print(("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}\n" +
               "TL_w: {:.2f}, TL_pos_w: {:.2f}, TL_neg_w: {:.2f}, TL_centroid_w: {:.2f}\n" + 
              "TL: {:.2f}, TL_pos: {:.2f}, TL_neg: {:.2f}, TL_centroid: {:.2f}").format(
            epoch+1, h[0], h_validation[0], h[1], h[2], h[3], h[4], h[5], h[6], h[7], h[8]))

/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch: 101 Loss: 0.02, Loss_test: 0.05
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.02, TL_pos: 0.02, TL_neg: 0.03, TL_centroid: 0.02
Epoch: 102 Loss: 0.02, Loss_test: 0.05
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.02, TL_pos: 0.02, TL_neg: 0.01, TL_centroid: 0.02
Epoch: 103 Loss: 0.04, Loss_test: 0.05
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.04, TL_pos: 0.04, TL_neg: 0.03, TL_centroid: 0.02
Epoch: 104 Loss: 0.06, Loss_test: 0.05
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.07, TL_pos: 0.07, TL_neg: 0.05, TL_centroid: 0.04
Epoch: 105 Loss: 0.03, Loss_test: 0.05
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.03, TL_neg: 0.02, TL_centroid: 0.03
Epoch: 106 Loss: 0.02, Loss_test: 0.05
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.02, TL_pos: 0.02, TL_neg: 0.01, TL_centroid: 0.02
Epoch: 107 Loss: 0.06, Loss_test: 0.05
TL_w: 1

Epoch: 152 Loss: 0.01, Loss_test: 0.04
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 153 Loss: 0.02, Loss_test: 0.04
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.02, TL_neg: 0.01, TL_centroid: 0.03
Epoch: 154 Loss: 0.03, Loss_test: 0.04
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.03, TL_neg: 0.02, TL_centroid: 0.03
Epoch: 155 Loss: 0.03, Loss_test: 0.04
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.02, TL_neg: 0.05, TL_centroid: 0.02
Epoch: 156 Loss: 0.01, Loss_test: 0.04
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 157 Loss: 0.03, Loss_test: 0.04
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.03, TL_neg: 0.02, TL_centroid: 0.04
Epoch: 158 Loss: 0.03, Loss_test: 0.04
TL_w: 1

Epoch: 203 Loss: 0.01, Loss_test: 0.03
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 204 Loss: 0.02, Loss_test: 0.03
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.01, TL_centroid: 0.04
Epoch: 205 Loss: 0.03, Loss_test: 0.03
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.03, TL_neg: 0.02, TL_centroid: 0.03
Epoch: 206 Loss: 0.03, Loss_test: 0.03
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.03, TL_neg: 0.01, TL_centroid: 0.04
Epoch: 207 Loss: 0.02, Loss_test: 0.03
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.02, TL_pos: 0.02, TL_neg: 0.01, TL_centroid: 0.02
Epoch: 208 Loss: 0.02, Loss_test: 0.03
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.01, TL_centroid: 0.04
Epoch: 209 Loss: 0.00, Loss_test: 0.03
TL_w: 1

Epoch: 254 Loss: 0.01, Loss_test: 0.03
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 255 Loss: 0.01, Loss_test: 0.03
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.01, TL_centroid: 0.02
Epoch: 256 Loss: 0.01, Loss_test: 0.03
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.01, TL_centroid: 0.01
Epoch: 257 Loss: 0.04, Loss_test: 0.03
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.03, TL_neg: 0.03, TL_centroid: 0.05
Epoch: 258 Loss: 0.04, Loss_test: 0.03
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.06, TL_pos: 0.05, TL_neg: 0.02, TL_centroid: 0.04
Epoch: 259 Loss: 0.03, Loss_test: 0.03
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.03, TL_neg: 0.02, TL_centroid: 0.05
=> result saved!
Epoch: 260 Loss: 0.04, Loss_t

...

Epoch: 662 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 663 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 664 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 665 Loss: 0.00, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.00, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 666 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.01, TL_centroid: 0.01
Epoch: 667 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 668 Loss: 0.01, Loss_test: 0.01
TL_w: 1

Epoch: 713 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 714 Loss: 0.02, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.03, TL_neg: 0.01, TL_centroid: 0.03
Epoch: 715 Loss: 0.00, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.00, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 716 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 717 Loss: 0.02, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.01, TL_centroid: 0.03
Epoch: 718 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 719 Loss: 0.01, Loss_test: 0.01
TL_w: 1

Epoch: 764 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 765 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.01, TL_centroid: 0.01
Epoch: 766 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 767 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 768 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.00, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 769 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.01, TL_centroid: 0.02
=> result saved!
Epoch: 770 Loss: 0.01, Loss_t

Epoch: 815 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.01, TL_centroid: 0.01
Epoch: 816 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 817 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 818 Loss: 0.00, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.00, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 819 Loss: 0.02, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.02, TL_centroid: 0.03
=> result saved!
Epoch: 820 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 821 Loss: 0.01, Loss_t

Epoch: 866 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.02, TL_pos: 0.02, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 867 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 868 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 869 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.02
=> result saved!
Epoch: 870 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 871 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.01, TL_centroid: 0.02
Epoch: 872 Loss: 0.01, Loss_t

In [44]:
len(result['train']), len(result['test'])

(900, 900)

In [45]:
encoded = model.get_layer('merge_features_in')
output = encoded.output
inputs = similarity_model.inputs[:-1]
encoded_anchor = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

In [47]:
SAVE_PATH.replace('@number_of_epochs@', str(epochs))

'bert_preprocessing_deepQL_weights_1000_feature1000epochs_64batch(openoffice)'

In [48]:
experiment.save_model(model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
"Model saved"

Saved model 'modelos/model_bert_preprocessing_deepQL_weights_1000_feature_1000epochs_64batch(openoffice).h5' to disk


'Model saved'

In [49]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, 1, encoded_anchor, issues_by_buckets, 
                                                               bug_train_ids, method='bert')
print(("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}\n" +
       "TL_w: {:.2f}, TL_pos_w: {:.2f}, TL_neg_w: {:.2f}, TL_centroid_w: {:.2f}\n" + 
        "TL: {:.2f}, TL_pos: {:.2f}, TL_neg: {:.2f}, TL_centroid: {:.2f}, " +
      "recall@25: {:.2f}").format(epoch+1, h[0], h_validation[0], h[1], h[2], h[3], h[4], h[5], h[6], h[7], h[8], recall))


Epoch: 900 Loss: 0.01, Loss_test: 0.01
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.01, TL_neg: 0.00, TL_centroid: 0.03, recall@25: 0.68


In [50]:
recall

0.68

In [51]:
exported_rank[:20]

['98306:88871,50853,33630,90791|42420:0.19545656442642212,43589:0.19489067792892456,88871:0.19465428590774536,69738:0.1913556456565857,95954:0.188298761844635,42080:0.1781025528907776,45914:0.16684597730636597,47977:0.16143375635147095,47227:0.15983110666275024,33257:0.15572553873062134,75829:0.15527796745300293,100549:0.15347659587860107,62750:0.153376042842865,84938:0.15250927209854126,37651:0.15245157480239868,63926:0.15216708183288574,54727:0.15016573667526245,93749:0.14937591552734375,98045:0.14786052703857422,53255:0.14612972736358643,58711:0.14493024349212646,98090:0.14355182647705078,96073:0.1432018280029297,79777:0.14253157377243042,58842:0.14032602310180664,58843:0.14032602310180664,80968:0.13810282945632935,90642:0.13569903373718262,48849:0.13400518894195557',
 '32771:32490,33548,32560,33879,32699|9917:0.30968791246414185,39209:0.30935412645339966,18023:0.30669575929641724,5412:0.29871535301208496,71665:0.29868197441101074,5080:0.296935498714447,3780:0.2968478798866272,17105

In [52]:
print("Total of queries:", len(retrieval.test))

Total of queries: 8265


In [53]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

'bert_preprocessing_deepQL_weights_1000_feature_1000epochs_64batch(openoffice)'

In [54]:
encoded_anchor.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 738)          0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 20)           0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 20)           0                                            
__________________________________________________________________________________________________
desc_segme

In [55]:
len(exported_rank)

11757

In [56]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/openoffice/bert/exported_rank_deepQL_weights_1000.txt'

In [57]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [58]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report

{'1 - recall_at_5': 0.56,
 '2 - recall_at_10': 0.61,
 '3 - recall_at_15': 0.64,
 '4 - recall_at_20': 0.66,
 '5 - recall_at_25': 0.68}